In [1]:
from birdy import WPSClient
import requests
import os
from datetime import datetime
from wps_tools import copy_http_content

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "osprey":
    os.chdir('../')

url = 'https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/osprey/wps'
osprey = WPSClient(url)

ImportError: cannot import name 'copy_http_content'

In [ ]:
# NBVAL_IGNORE_OUTPUT
thunderbird.update_metadata?

In [ ]:
# run update_metadata
updates = resource_filename("tests", "metadata-conversion/simple-conversion.yaml")
netcdf = "https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/dodsC/datasets/TestData/gdd_annual_CanESM2_rcp85_r1i1p1_1951-2100.nc"
output = thunderbird.update_metadata(updates = updates, netcdf = netcdf)

In [ ]:
output_data = Dataset(make_tmp_copy(output))